In [1]:
import numpy as np
import pandas as pd

In [6]:
credits = pd.read_csv("/home/supernova/Placement/Projects/Movie Recommender using Iterative RSVD/tmdb_5000_credits.csv")
movies = pd.read_csv("/home/supernova/Placement/Projects/Movie Recommender using Iterative RSVD/tmdb_5000_movies.csv")
ratings = pd.read_csv("/home/supernova/Placement/Projects/Movie Recommender using Iterative RSVD/ratings_small.csv")

In [7]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [9]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
credits.rename(columns={'movie_id':'movie'}, inplace=True)
movies.rename(columns={'id':'movie'}, inplace=True)
ratings.rename(columns={'movieId':'movie', "userId":"user"}, inplace=True)

In [12]:
data = pd.merge(credits, movies, on="movie")
df = data.merge(ratings, on="movie")

In [16]:
ratings = ratings.loc[:,["user","movie","rating"]]
ratings

,user,movie,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [ ]:
df.drop(['status', 'title_x', 'title_y'], axis=1, inplace=True)
df.head(1)

,movie,cast,crew,budget,genres,homepage,keywords,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,user,rating,timestamp
0,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",6.9,4500,39,4.0,832523436


In [21]:
df.vote_average.mean()

6.802310053308922

In [22]:
# We need movie id and title
df = df.loc[:,["movie","original_title","user","vote_average"]]

# Re-writing the movie columns into number from 1-857 one number for each "original_title"
df["movie"] = df["original_title"].astype('category').cat.codes
df["user"] = df["user"].astype('category').cat.codes

# df['user'].unique().shape
df['user'] +=1
df['movie'] +=1
print(df["movie"].max(), df["movie"].min())

856 1


In [23]:
# selecting movie, user and vote_average colmns
df1 = df.loc[:,["movie","user","vote_average"]]
df1.rename(columns={'vote_average':'rating'}, inplace=True)
df1.to_csv("Processed_data_tmdb.csv")

In [24]:
# Movie name vs movieId table
movie_names = df[["movie","original_title"]]
movie_names.sort_values("movie", inplace=True)
# Removing duplicates
movie_names.drop_duplicates(keep="first", inplace=True)
movie_names.head(10)

movie_names.to_csv("movie_names_tmdb.csv")

/tmp/ipykernel_41140/3487381255.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_names.sort_values("movie", inplace=True)
/tmp/ipykernel_41140/3487381255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_names.drop_duplicates(keep="first", inplace=True)
